# Transfer Learning with TensorFlow Part 3: Scaling up (Food Vision Mini)

Using 101 classes. 

Goal: Beat original food 101 paper accuracy : 50.76%

In [12]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [13]:
## Creating helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-01-17 04:21:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-17 04:21:11 (23.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [14]:
# Import helper functions
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,compare_historys,walk_through_dir

## 101 Food Classes: working with less data



In [15]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data('101_food_classes_10_percent.zip')

train_dir = '101_food_classes_10_percent/train/'
test_dir = '101_food_classes_10_percent/test/'

--2023-01-17 04:21:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  30.7MB/s    in 54s     

2023-01-17 04:22:06 (28.5 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [16]:
# How many images/classes
walk_through_dir('101_food_classes_10_percent')

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/prime_rib'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/strawberry_shortcake'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/crab_cakes'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/dumplings'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/clam_chowder'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/french_onion_soup'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/club_sandwich'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/peking_duck'.
There are 0 directories and 75 im

In [17]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224,224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode='categorical',
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                               label_mode='categorical',
                                                                               image_size=IMG_SIZE,
                                                                               shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big dog model with trainsfer learning on 10% of 101 food classes

Steps we are going to take

1. Create a ModelCheckpoint callback
2. Create a data augmentation layer
3. Build a headless(no top layers) Functional EfficientNetB0 backboned-model (Will create our own output layer)
4. Compile 
5. Feature extract for 5 epochs (validate on 15% of testing data for each epoch)


In [18]:
# Create checkpoint callback
checkpoint_path = '101_classes_10_percent_data_model_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

In [19]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
    # preprocessing.Rescaling(1/255.) 
],name='data_augmentation')


In [20]:
# Setup the bease model and freeze its layers (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with tranable top layers
inputs = layers.Input(shape=(224,224,3),name='input_layer')
x = data_augmentation(inputs)
x = base_model(x,training=False) 
x = layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),activation='softmax',name='output_layer')(x)
model = tf.keras.Model(inputs,outputs)

16705208/16705208 [==============================] - 1s 0us/step


In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_avg_pool_layer (Glob  (None, 1280)             0         
 alAveragePooling2D)                                             
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainab

In [22]:
# Compile the model
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
from tensorflow.python import test
# Fit the model
history_0 = model.fit(train_data_all_10_percent,
                      epochs = 5,
                      validation_data = test_data,
                      validation_steps = int(0.15*len(test_data)),
                      callbacks = [checkpoint_callback])

Epoch 1/5
125/237 [==============>...............] - ETA: 4:40 - loss: 3.8516 - accuracy: 0.1828

In [ ]:
# Evaluate the model
feature_extraction_results = model.evaluate(test_data)
feature_extraction_results

In [ ]:
plot_loss_curves(history_0)

## Fine-tuning

In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable = True

# Refreeze every layer except the last 5
for layer in base_model.layers[:-5]:
  layers.trainable = False

In [ ]:
# Recompile model with lowe learning 
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(lr=0.0001), # reduce by 10x
    metrics = ['accuracy']
)

In [ ]:
# What layers in the model are trainable?
for layer in model.layers:
  print(layer.name,layer.trainable)

In [ ]:
# Check which layers are trainable in our base model
for layer_number,layer in enumerate(model.layers[2].layers):
  print(layer_number,layer.name,layer.trainable)

In [ ]:
# Fine-tune for 5 more epochs
fine_tune_epochs = 10 # model has already done 5 epochs (feature extraction)

# Fine-tune our model
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15*len(test_data)),
                                                     initial_epoch=history_0.epoch[-1])

In [ ]:
# Evaluate on the whole test data
all_classes_10_percent_fine_tune_results = model.evaluate(test_data)
all_classes_10_percent_fine_tune_results

In [ ]:
# Compare the histories of feature extraction model with fine-tuning model
compare_historys(original_history=history_0,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)

## Saving and loading our model

To use our model in an external application, we will need to save it and export it somewhere

In [ ]:
# Save our fine-tuned model
model.save('drive/MyDrive/Tensorflow_Certificate/101_food_classes_10_percent_saved_big_dog_model')

In [ ]:
# Load saved model
loaded_model = tf.keras.models.load_model('drive/MyDrive/Tensorflow_Certificate/101_food_classes_10_percent_saved_big_dog_model')

In [ ]:
# Evaluate loaded model and compare to previous model
loaded_model_results = loaded_model.evaluate(test_data)
loaded_model_results

In [ ]:
# The result above should be similar to results below
all_classes_10_percent_fine_tune_results

## Evlauating the performance of the big dog model across all different classes

Let's make some predictioins, visualize them and then later find out which predictions were the 'most' wrong.

In [ ]:
import tensorflow as tensorflow
# Donwload pretrained model pretrained earlier
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [ ]:
# Unzip the data
unzip_data('06_101_food_class_10_percent_saved_big_dog_model.zip')

In [ ]:
# Loading the model
model = tf.keras.models.load_model('06_101_food_class_10_percent_saved_big_dog_model')

In [ ]:
# Evaluate the model (the one we just downloaded on test data)
results_downloaded_model = model.evaluate(test_data)
results_downloaded_model

## Making predictions with our trained model

In [ ]:
# Make predictions
preds_probs = model.predict(test_data,verbose=1) # set verbosity to see how long is left

In [ ]:
# How many predictions are there?
len(preds_probs)

In [ ]:
# What's the shape of our predictions?
preds_probs.shape

In [ ]:
# Let's see what the first 10 predictions look like
preds_probs[:10]

In [ ]:
# What does the first prediction probabitlity array look like
preds_probs[0] , len(preds_probs[0]), sum(preds_probs[0])

In [ ]:
print(f'Number of prediction probabilities for sample 0: {len(preds_probs[0])}')
print(f'What prediction probability sample 0 looks like:\n {preds_probs[0]}')
print(f'The clas with the highest predicted probability by the model for sample 0: {preds_probs[0].argmax()}')

In [ ]:
# Get the pred classes of each label
pred_classes = preds_probs.argmax(axis=1)

#How do thet look?
pred_classes[:10]

In [ ]:
# How many pred classes do we have?
len(pred_classes)

In [ ]:
# to get our test labels, we need to unravel our test_data BatchDataset
y_labels = []
for images,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax()) # currently test labels look like: [0,0,0,1....,0,0]. we want the index value where the '1' occurs
y_labels[:10] # look at the first 10

In [ ]:
# How many y_labels are there?
len(y_labels)

## Evaluating our model's predictions

One way to check that our model's predictions array is in the same order as our test labels array is to find the accuracy score

In [ ]:
results_downloaded_model

In [ ]:
# Let;s try scikit-learn's accuracy score function and see what it comes up with
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_true=y_labels,
                                  y_pred=pred_classes)
sklearn_accuracy

In [ ]:
# Does this metric come close to our model's evaluate results
import numpy as np
np.isclose(results_downloaded_model[1],sklearn_accuracy)

## Making a Confusion Matrix

In [ ]:
from helper_functions import make_confusion_matrix

In [ ]:
# Get a list of class names
class_names = test_data.class_names
class_names[:10]

In [ ]:
# We need to make some changes to our make_confusion_function to ensure the x-labels print vertically

In [ ]:
import include_tools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confustion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Changed (plot x-labels vertically)
  plt.xticks(rotation=70,fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")
  

In [ ]:
make_confusion_matrix(y_true=y_labels,
                       y_pred=pred_classes,
                       classes=class_names,
                       figsize=(100,100),
                       text_size=20)

## Let's keep the evaluation train going, time for a classification report

Sklearn has a helpful function for acquiring many different classification metrics per class (eg. precision, recall and f1) called [classification_report](https:// [link text](https://)) let's try it out.

In [ ]:
from sklearn.metrics import classification_report
print(classificaiton_report(y_true=y_labels,
                            y_pred=pred_class)

The numbers above gives a class-by-class evaluation. 

How about we create a visualization to get a better understanding. 

In [ ]:
# Get a dictionary of classification report
classification_report_dict = classification_report(y_labels,pred_classes,output_dict=True)
classification_report_dict

Let's plot all of our classes F1-scores

In [ ]:
# Create empty dictionary 
class_f1_scores = {}
# Loop through classification report dictionary items
for k,v in classification_report_dict.items():
  if k = 'accuracy': # stop once we get to accuracy key
    break
  else:
    # Add class names and f1-scores to new dictionary
    class_f1_scores[class_names[int(k)]] = c['f1-score']

In [ ]:
# Turn f1-scores into dataframe for visualization
import pandas as pd
f1_scores = pd.DataFrame({'class_names' : list(class_f1_scores.keys()),
                          'f1-score' : list(class_f1_score.values())}).sort_values('f1-score',ascending=False)

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(12,25))
scores = ax.barh(range(len(f1_scores)),f1_scores['f1-score'].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(f1_scores['class_names'])
ax.set_xlabel('F1-score')
ax.set_title('F1-Score for 101 Different Food Classes')
ax.invert_yaxis();

## Visualization predictions on custom images

Now, this is the real test, how does our model go on food images not even in our test dataset(images of our own).